- 분석 준비
- 데이터셋 구조 파악 (컬럼 설명)
* user_id: 사용자 ID
* timestamp: 테스트 실행 시간
* group: 테스트 그룹 (A/B)
* version: 알고리즘 버전 (v1/v2)
* conversion: 전환 여부 (0/1)
* revenue: 수익 금액
- pandas로 데이터셋 로드 및 전처리



### A/B 테스트 통계적 분석 수행

#### 꼭 담겨야할 분석 내용 (요구사항 정리)

1. 데이터셋 정보 확인

2. 데이터셋 기술통계량 확인
3. 데이터셋 pandas 전처리 (결측치 처리, 데이터 타입 변환, 데이터 정렬 등)
4. A/B 테스트 가설 설정
5. 전환율(conversion rate) 분석
6. 수익(revenue) 분석
7. 통계적 유의성 검정 (카이제곱 검정, t-test 등)
8. 시각화 (plotly 사용)
9. 그룹별 전환율 비교

10. 그룹별 수익 비교
11. 시간에 따른 전환율/수익 추이
12. 분석 결과 해석 및 결론 도출

#### 1. 데이터셋 정보 확인

확인할 정보
- 초기 데이터셋 info 확인
- 컬럼명, 데이터 타입 확인하기
- 결측치, 이상치 존재 여부 확인

In [1]:
import pandas as pd

df = pd.read_csv('../datasets/algorithm_ab_test.csv')
df.info()
print(df.algorithm.unique(), df.grp.unique(), df.converted.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441445 entries, 0 to 441444
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   uid        441445 non-null  int64 
 1   ts         441445 non-null  object
 2   grp        441445 non-null  object
 3   algorithm  441445 non-null  object
 4   converted  441445 non-null  int64 
 5   amount     441445 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 20.2+ MB
['v1' 'v2' 'v3'] ['A' 'B' 'C'] [0 1]


In [2]:
df.isnull().sum()

uid          0
ts           0
grp          0
algorithm    0
converted    0
amount       0
dtype: int64

#### 초기 데이터셋 확인하기
- grp(group) : 그룹 정보 (A, B, C)
- algorithm : 알고리즘 정보 (V1, V2, V3)
- amount(revenue) : 수익 (거래 금액)
- ts(timestamp) : 거래 시간 object -> datetime 타입으로 변환 필요
- converted : 전환 여부 (0, 1)
- 결측치 없음

1-1. 컬럼 `'ts'` datetime 타입으로 변환하기

In [3]:
df['ts'] = pd.to_datetime(df['ts'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441445 entries, 0 to 441444
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   uid        441445 non-null  int64         
 1   ts         441445 non-null  datetime64[ns]
 2   grp        441445 non-null  object        
 3   algorithm  441445 non-null  object        
 4   converted  441445 non-null  int64         
 5   amount     441445 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 20.2+ MB


1-2. 그룹별 개수, 알고리즘별 개수 확인하기

In [4]:
df['grp'].value_counts(),df['algorithm'].value_counts()

(grp
 B    147276
 A    147202
 C    146967
 Name: count, dtype: int64,
 algorithm
 v1    147239
 v2    147239
 v3    146967
 Name: count, dtype: int64)

1-3. 그룹별 개수, 알고리즘별 개수 시각화 진행

In [5]:
import plotly.express as px


# 그룹별 개수
fig = px.bar(df['grp'].value_counts().sort_index(), 
             labels={'grp':'Group'}, 
             title='그룹별 샘플 수',
             text_auto='.0f')
fig.show()

# 알고리즘별 개수
fig = px.bar(df['algorithm'].value_counts(), 
            #  labels={'':'Algorithm'}, 
             title='알고리즘별 샘플 수',
             text_auto='.0f')
fig.show()

#### 2. 데이터셋 기술통계량 확인

2-1. 수치형 컬럼 기술통계 describe() 확인하기

In [6]:
df.describe()

,uid,ts,converted,amount
count,441445.000000,441445,441445.000000,441445.000000
mean,647245.301326,2025-01-13 13:45:07.846213120,0.119754,8998.378613
min,100002.000000,2025-01-02 13:32:15.234051,0.000000,0.000000
25%,498388.000000,2025-01-08 02:15:34.926518016,0.000000,0.000000
50%,709298.000000,2025-01-13 13:30:23.756212992,0.000000,12103.000000
75%,827644.000000,2025-01-19 01:49:07.722962944,0.000000,13162.000000
max,946122.000000,2025-01-24 13:50:19.152664,1.000000,17778.000000
std,230428.065706,NaN,0.324675,5877.227027


2-2. A/B TEST 진행 전 데이터셋에서 전처리 필요한 부분 확인하기

In [7]:
# 그룹-알고리즘 조합별 샘플 수 확인(그룹과 알고리즘의 관계 확인하기 위해)
pd.crosstab(df['grp'], df['algorithm'])

algorithm,v1,v2,v3
grp,,,
A,145274,1928,0
B,1965,145311,0
C,0,0,146967


#### A/B 테스트 실험 설계에 맞지 않기 때문에 데이터셋 전처리 진행함.  
WHY?   
- 알고리즘과 그룹별 간 관계에서 대부분의 데이터는  
    - **'v1' 알고리즘은 'A' 그룹에 속함.**  
    - **'v2' 알고리즘은 'B' 그룹에 속함.**  
    - **'v3' 알고리즘은 'C' 그룹에 속함.**  
여기에 충족하지 않는 데이터는 (교차 샘플) 약 3000개 정도 존재함.  
(그룹 A - v2 알고리즘: 1928개, 그룹 B - v1 알고리즘 1965개)  

또한, 여기서 그룹 C와 v3 알고리즘에 해당하는 데이터는 실험 설계의 일관성과 가설 검정의 명확성을 위해 분석에서 제외함

#### 결론
- A그룹의 v2, B그룹의 v1 샘플(각각 1928, 1965건)
- C 그룹 및 v3 알고리즘에 해당하는 **모든 데이터는 분석에서 제외**함.

**A그룹 (v1 알고리즘)**  
**B그룹 (v2 알고리즘)**

두 집단을 대상으로 A/B TEST 수행합니다.


#### 3. 데이터셋 pandas 전처리

- 해당 조건에 맞는 데이터만 필터링 진행

In [8]:
cond1 = (df['algorithm'] == 'v1') & (df['grp'] == 'A')
cond2 = (df['algorithm'] == 'v2') & (df['grp'] == 'B')


# df[cond1]
filter_df = df[cond1 | cond2]
filter_df
print(df.shape, filter_df.shape) # 약 3000개 + 140000개 샘플 제거

(441445, 6) (290585, 6)


#### 4. A/B 테스트 가설 설정

- 타겟변수가 될 `전환율`, `수익`에 대한 귀무, 대립가설 작성하기
1. 전환율
    - 귀무가설: 두 알고리즘(그룹) 간 전환율 차이가 없다.
    - 대립가설: 두 알고리즘(그룹) 간 전환율 차이가 있다.
2. 수익
    - 귀무가설: 두 알고리즘의 간 평균 수익 차이가 없다.
    - 대립가설: 두 알고리즘 간 평균 수익 차이가 있다.

#### 5. 전환율 분석

- 그룹별 전환율 계산
- 전환율 분포 시각화 진행

- 범주형 데이터인 전환여부는 카이제곱으로 검증한다.
- 수치형 데이터인 수익은 평균 차이 검정(t-test)으로 검증한다.

In [9]:
# - 그룹별 전환/비전환 빈도표 생성

converted_df = pd.crosstab(filter_df['grp'], filter_df['converted'])
converted_df

converted,0,1
grp,,
A,127785,17489
B,128047,17264


In [10]:
# 카이제곱 검정을 통해 두 그룹의 전환율 차이가 유의미한지 확인해보기

from scipy.stats import chi2_contingency


chi2, p, dof, expected = chi2_contingency(converted_df)
print(f"카이제곱 검정 통계량: {chi2:.4f} p-value: {p:.4f}")

카이제곱 검정 통계량: 1.7054 p-value: 0.1916


In [11]:
# 그룹별 전환율 시각화 진행

import plotly.graph_objects as go
import plotly.express as px

# 전환율 계산
conversion_rates = converted_df.div(converted_df.sum(axis=1), axis=0)[1]

# 막대 그래프 생성
fig = go.Figure()

# 전환율 막대 그래프 추가
fig.add_trace(go.Bar(
    x=conversion_rates.index,
    y=conversion_rates.values,
    text=[f'{rate:.2%}' for rate in conversion_rates.values],
    textposition='auto',
    name='전환율'
))

# 레이아웃 설정
fig.update_layout(
    title='그룹별 전환율 비교',
    xaxis_title='그룹',
    yaxis_title='전환율',
    yaxis_tickformat='.1%',
    showlegend=False,
    template='plotly_white'
)

# 신뢰구간 표시를 위한 오차 막대 추가
fig.add_trace(go.Scatter(
    x=conversion_rates.index,
    y=conversion_rates.values,
    mode='markers',
    marker=dict(
        size=10,
        symbol='diamond'
    ),
    error_y=dict(
        type='data',
        array=[0.01, 0.01],  # 95% 신뢰구간
        visible=True
    ),
    name='신뢰구간'
))

fig.show()


In [12]:
# 카이제곱 검정 시각화
import numpy as np
from scipy import stats
import plotly.graph_objects as go

# 관찰된 빈도수 행렬
observed = pd.crosstab(filter_df['grp'], filter_df['converted']).values

# 기대빈도 계산
row_totals = observed.sum(axis=1)
col_totals = observed.sum(axis=0)
total = observed.sum()
expected = np.outer(row_totals, col_totals) / total

# 카이제곱 통계량 계산
chi2_stat = np.sum((observed - expected)**2 / expected)
p_value = 1 - stats.chi2.cdf(chi2_stat, df=1)

# 카이제곱 분포 시각화
x = np.linspace(0, 10, 1000)
chi2_pdf = stats.chi2.pdf(x, df=1)

fig = go.Figure()

# 카이제곱 분포 곡선
fig.add_trace(go.Scatter(
    x=x,
    y=chi2_pdf,
    mode='lines',
    name='카이제곱 분포',
    line=dict(color='blue', width=2)
))

# 관찰된 카이제곱 통계량 표시
fig.add_vline(x=chi2_stat, line_dash="dash", line_color="green",
              annotation_text=f"관찰된 통계량 ({chi2_stat:.2f})")

# 임계값 표시 (유의수준 0.05)
critical_value = stats.chi2.ppf(0.95, df=1)
fig.add_vline(x=critical_value, line_dash="dash", line_color="red",
              annotation_text="임계값 (3.84)")

# 레이아웃 설정
fig.update_layout(
    title=f'카이제곱 검정 결과 (p-value: {p_value:.4f})',
    xaxis_title='카이제곱 통계량',
    yaxis_title='확률 밀도',
    template='plotly_white',
    showlegend=True
)

fig.show()

# 검정 결과 출력
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"임계값 (α=0.05): {critical_value:.4f}")



카이제곱 통계량: 1.7203
p-value: 0.1897
임계값 (α=0.05): 3.8415


- pvalue 값이 0.1897이므로 귀무가설을 기각할 수 없다.
- 따라서 두 그룹 간 전환율 차이는 통계적으로 유의하지 않다고 볼 수 있다.

#### 6. 수익 분석
- 그룹별 수익 계산
- 수익 분포 시각화 진행

- 수익이 존재하는 데이터는 즉, 전환이 이루어진 경우만 존재함
    - **전환이 이루어지지 않은 것은 애초에 수익이 발생하지 않음, 전처리 해주기**

- 따라서 전환이 된 컬럼만 필터링 후 수익분석 진행

In [13]:
revenue_df = filter_df[filter_df['converted'] == 1]
revenue_df

,uid,ts,grp,algorithm,converted,amount
4,865098,2025-01-21 01:52:26.210827,A,v1,1,13058
6,679810,2025-01-19 03:26:46.940749,B,v2,1,12538
8,817478,2025-01-04 17:58:08.979471,B,v2,1,12330
9,839908,2025-01-15 18:11:06.610965,B,v2,1,12469
15,644337,2025-01-22 02:05:21.719434,A,v1,1,13439
...,...,...,...,...,...,...
294396,838716,2025-01-15 09:56:31.455023,B,v2,1,14337
294405,712340,2025-01-11 10:34:30.176801,A,v1,1,12214
294420,795865,2025-01-09 01:06:58.299207,A,v1,1,13818
294430,733994,2025-01-21 17:54:08.810964,B,v2,1,12501


In [14]:
# A, B 그룹만 필터링된 데이터 사용
a_amount = revenue_df[revenue_df['grp'] == 'A']['amount']
b_amount = revenue_df[revenue_df['grp'] == 'B']['amount']
a_amount, b_amount

(4         13058
 15        13439
 28        11264
 36        12044
 43        11777
           ...  
 294383    12620
 294385    12917
 294405    12214
 294420    13818
 294443    12464
 Name: amount, Length: 17489, dtype: int64,
 6         12538
 8         12330
 9         12469
 17        15515
 26        12506
           ...  
 294371    13949
 294382    12121
 294388    13363
 294396    14337
 294430    12501
 Name: amount, Length: 17264, dtype: int64)

- 평균과 분산 확인하기
- 두 그룹 간 데이터 분포 확인하기

In [15]:
print(f"{np.mean(a_amount):.2f}", f"{np.std(a_amount):.2f}")
print(f"{np.mean(b_amount):.2f}", f"{np.std(b_amount):.2f}")


12537.36 1099.26
12745.02 1159.91


In [17]:
import plotly.express as px
from scipy.stats import t

plot_df = pd.concat([
    pd.DataFrame({'Group': 'A', 'Amount': a_amount}),
    pd.DataFrame({'Group': 'B', 'Amount': b_amount})
])

fig = px.box(plot_df, x='Group', y='Amount', points='all', title='그룹별 수익 분포(박스플롯)')

# 평균 + 신뢰구간 추가
for group in ['A', 'B']:
    mean = np.mean(revenue_df[revenue_df['grp'] == group]['amount'])
    group_data = revenue_df[revenue_df['grp'] == group]['amount']
    n = len(group_data)
    std_err = np.std(group_data, ddof=1) / np.sqrt(n)
    t_critical = t.ppf(0.975, df=n-1)  # 95% 신뢰구간
    ci = t_critical * std_err
    fig.add_scatter(
        x=[group], y=[mean],
        mode='markers',
        marker=dict(color='red', size=10),
        error_y=dict(type='data', array=[ci], visible=True),
        name=f'{group} 평균+신뢰구간'
)

fig.update_yaxes(tickformat=',d')  # y축 숫자 정수로 표기
fig.show()

- 두 그룹 간 평균은 약 200정도 차이가 발생했다.
- t 검정을 통해 두 그룹 간 평균의 차이가 유의미한지 검증한다.

In [18]:
from scipy.stats import ttest_ind

t_stat, p_value = ttest_ind(a_amount.tolist(), b_amount.tolist(), equal_var=False) # 등분산성 가정이 어려우면 equal_var=False


print(f"\n연속형 지표 (평균 매출액) t-검정 결과:")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.3e}")

alpha = 0.05
if p_value < alpha:
    print(f"결론: p-value ({p_value:.3e}) < 유의수준 ({alpha}).")
    print("      귀무 가설을 기각합니다. 두 그룹 간 평균 매출액에 통계적으로 유의미한 차이가 존재합니다.")
else:
    print(f"결론: p-value ({p_value:.3e}) >= 유의수준 ({alpha}).")
    print("      귀무 가설을 기각하지 못합니다. 두 그룹 간 평균 매출액에 통계적으로 유의미한 차이가 있다고 보기 어렵습니다.")


연속형 지표 (평균 매출액) t-검정 결과:
T-statistic: -17.1253
P-value: 1.790e-65
결론: p-value (1.790e-65) < 유의수준 (0.05).
      귀무 가설을 기각합니다. 두 그룹 간 평균 매출액에 통계적으로 유의미한 차이가 존재합니다.


#### 카이제곱, t-검정 결과
t 검정 결과 p-value값이 유의수준보다 매우 아래에 있으므로 두 집단 간 차이가 존재한다는 검정 결과가 나왔다.
두 그룹간 평균 매출액에 통계적으로 유의미한 차이가 발생한다.

다만 매출액은 통계적으로 유의미한 차이를 보였으나, 전환율은 유의미한 차이가 없었다.

#### 7. 시간에 따른 전환율, 수익 분포 확인

In [19]:
df.ts.max(), df.ts.min()

(Timestamp('2025-01-24 13:50:19.152664'),
 Timestamp('2025-01-02 13:32:15.234051'))

In [20]:
df['date'] = df['ts'].dt.date
df.drop('ts', axis=1, inplace=True)
df.set_index('date', inplace=True)
df.sort_values(by='date', inplace=True)
df.head()



,uid,grp,algorithm,converted,amount
date,,,,,
2025-01-02,769402,A,v1,0,0
2025-01-02,795396,B,v2,1,13307
2025-01-02,835472,B,v2,0,11936
2025-01-02,866065,A,v1,0,0
2025-01-02,652078,A,v1,0,0


In [21]:
df.groupby(['date','grp']).mean('converted')

uid  converted        amount
date       grp                                        
2025-01-02 A    790236.012785   0.125086   1571.643746
           B    789888.334372   0.120457  12736.237799
           C    367657.651139   0.126639  12737.975500
2025-01-03 A    790439.586176   0.113704   1419.234291
           B    789934.781073   0.113562  12746.877943
...                       ...        ...           ...
2025-01-23 B    789589.456302   0.120987  12759.848395
           C    365794.427033   0.123358  12742.104914
2025-01-24 A    789262.987930   0.118342   1484.291000
           B    788172.027904   0.120472  12747.195063
           C    360984.212049   0.123389  12730.486714

[69 rows x 3 columns]

In [23]:
# 그룹 C를 제외하고 A, B 그룹만 선택
df_ab = df[df['grp'].isin(['A', 'B'])]
df_ab


,uid,grp,algorithm,converted,amount
date,,,,,
2025-01-02,769402,A,v1,0,0
2025-01-02,795396,B,v2,1,13307
2025-01-02,835472,B,v2,0,11936
2025-01-02,866065,A,v1,0,0
2025-01-02,652078,A,v1,0,0
...,...,...,...,...,...
2025-01-24,701913,A,v1,0,0
2025-01-24,870385,B,v2,0,12400
2025-01-24,928681,A,v1,0,0


In [30]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('시간에 따른 전환율 변화 (그룹 A vs B)', '시간에 따른 평균 매출액 변화 (그룹 A vs B)'),
    vertical_spacing=0.1
)

# 전환율 데이터 준비
conversion_by_date = df_ab.groupby(['date', 'grp'])['converted'].mean().unstack()

# 전환율 그래프 추가
fig.add_trace(
    go.Scatter(x=conversion_by_date.index, y=conversion_by_date['A'], 
               mode='lines+markers', name='그룹 A', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=conversion_by_date.index, y=conversion_by_date['B'], 
               mode='lines+markers', name='그룹 B', line=dict(color='red')),
    row=1, col=1
)


amount_by_date = df_ab[df_ab['amount'] > 0].groupby(['date', 'grp'])['amount'].mean().unstack()

# 평균 매출액 그래프 추가
fig.add_trace(
    go.Scatter(x=amount_by_date.index, y=amount_by_date['A'], 
               mode='lines+markers', name='그룹 A', line=dict(color='blue'), showlegend=False),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=amount_by_date.index, y=amount_by_date['B'], 
               mode='lines+markers', name='그룹 B', line=dict(color='red'), showlegend=False),
    row=2, col=1
)

# 레이아웃 설정
fig.update_layout(
    height=800,
    title_text="그룹 A vs B 시간별 지표 변화",
    title_x=0.5
)

fig.update_yaxes(title_text="전환율", row=1, col=1)
fig.update_yaxes(title_text="평균 매출액", row=2, col=1)

fig.show()

# 통계 요약 출력
print("\n=== 그룹 A vs B 시간별 통계 요약 ===")
print("\n전환율 통계:")
print(conversion_by_date.describe())

print("\n평균 매출액 통계:")
print(amount_by_date.describe())



=== 그룹 A vs B 시간별 통계 요약 ===

전환율 통계:
grp            A          B
count  23.000000  23.000000
mean    0.120477   0.118971
std     0.003826   0.004099
min     0.113121   0.111398
25%     0.118633   0.116038
50%     0.120545   0.118260
75%     0.122406   0.120906
max     0.126903   0.127281

평균 매출액 통계:
grp               A             B
count     23.000000     23.000000
mean   12538.426884  12744.250610
std       40.404541     13.235501
min    12465.120823  12716.062621
25%    12514.142178  12735.304506
50%    12534.325377  12746.877943
75%    12566.479403  12749.979193
max    12627.868938  12776.611773


수치적으로 봐도 A,B의 차이가 조금 있는 것을 확인할 수 있다